In [1]:
import json

from ast import literal_eval

import pandas as pd

from openai import OpenAI
from tqdm import tqdm

In [ ]:
API_KEY = "ENTER-YOUR-API-KEY"
BASE_URL = "ENTER-YOUR-PROVIDER-URL"

In [3]:
client = OpenAI(api_key=API_KEY, base_url=BASE_URL)

In [4]:
response = client.responses.create(
    model="gpt-4o",
    instructions="You are a coding assistant that talks like a pirate.",
    input="How do I check if a Python object is an instance of a class?",
)

print(response.output_text) 

Arrr, if ye be wantin' to check if a Python object be an instance of a class, ye can use the `isinstance()` function. Here be how ye do it:

```python
# Define yer class
class PirateShip:
    pass

# Create an object of the class
black_pearl = PirateShip()

# Check if 'black_pearl' be an instance of 'PirateShip'
if isinstance(black_pearl, PirateShip):
    print("Aye, she be a fine PirateShip!")
else:
    print("Nay, she be no PirateShip!")
```

Use `isinstance(object, class)` to see if yer object be sailin' under a particular class flag. Yarrr!


In [ ]:
with open("../data_input/EgeEvalDataset.json", 'r', encoding="utf8") as inp:
    ege_eval_dataset = json.load(inp)

In [6]:
ege_eval_dataset[0]

{'index': 0,
 'reading_text': ' На задворках нашего села стояло на сваях длинное помещение из досок. Я первый раз в жизни здесь услышал музыку — скрипку. На ней играл Вася-поляк. О чём мне рассказывала музыка? О чём-то очень большом. На что же это жаловалась она, на кого гневалась? Тревожно и горько мне. Хочется заплакать, оттого что мне жалко самого себя, жалко тех, что спят непробудным сном на кладбище! Вася, не переставая играть, говорил: « Эту музыку написал человек, которого лишили самого дорогого. Если у человека нет матери, нет отца, но есть родина, он ещё не сирота. Всё проходит: любовь, сожаление о ней, горечь утрат, даже боль от ран, — но никогда не проходит и не гаснет тоска по родине. Эту музыку написал мой земляк Огинский. Написал на границе, прощаясь с родиной. Он посылал ей последний привет. Давно уже нет композитора на свете, но боль его, тоска его, любовь к родной земле, которую никто не может отнять, жива до сих пор». «Спасибо вам, дяденька», — прошептал я. «За что, м

In [7]:
def generate_prompt(example: dict) -> tuple[str, str]:
    instruction = "На вход поступают текст для чтения, тестовый вопрос к нему и правильный ответ на вопрос в формате: " + \
                  "Текст для чтения ВОПРОС: Текст вопроса ПРАВИЛЬНЫЙ ОТВЕТ: Текст правильного ответа. " + \
                  "Сгенерируй 3 неправильных ответа на вопрос так, чтобы они могли быть использованы в экзаменационном тестировании. " + \
                  "Отделяй полученные неправильные варианты друг от друга при помощи символа переноса строки, не используя нумерацию."
    prompt = f'{example["reading_text"]} ВОПРОС: {example["question"]} ПРАВИЛЬНЫЙ ОТВЕТ: {example["right_answer"]} НЕПРАВИЛЬНЫЕ ВАРИАНТЫ ОТВЕТА:'
    return instruction, prompt

In [8]:
instructions, prompt = generate_prompt(ege_eval_dataset[0])
print(instructions)
print(prompt)

На вход поступают текст для чтения, тестовый вопрос к нему и правильный ответ на вопрос в формате: Текст для чтения ВОПРОС: Текст вопроса ПРАВИЛЬНЫЙ ОТВЕТ: Текст правильного ответа. Сгенерируй 3 неправильных ответа на вопрос так, чтобы они могли быть использованы в экзаменационном тестировании. Отделяй полученные неправильные варианты друг от друга при помощи символа переноса строки, не используя нумерацию.
 На задворках нашего села стояло на сваях длинное помещение из досок. Я первый раз в жизни здесь услышал музыку — скрипку. На ней играл Вася-поляк. О чём мне рассказывала музыка? О чём-то очень большом. На что же это жаловалась она, на кого гневалась? Тревожно и горько мне. Хочется заплакать, оттого что мне жалко самого себя, жалко тех, что спят непробудным сном на кладбище! Вася, не переставая играть, говорил: « Эту музыку написал человек, которого лишили самого дорогого. Если у человека нет матери, нет отца, но есть родина, он ещё не сирота. Всё проходит: любовь, сожаление о ней, 

In [9]:
def ask_model(
    example: dict, model: str="gpt-4o", temperature: float=None
) -> str:
    instructions, prompt = generate_prompt(example)
    if not temperature:
        response = client.responses.create(
            model=model, instructions=instructions, input=prompt
        )
    else:
        response = client.responses.create(
            model=model, instructions=instructions,
            input=prompt, temperature=temperature
        )
    return response.output_text

In [10]:
print(ask_model(ege_eval_dataset[0]))

Вася-поляк играл на органе, а не на скрипке.

Автор чувствовал радость и счастье, слушая музыку Огинского.

Музыка Огинского никак не повлияла на воспоминания автора.


In [11]:
def get_model_answers(input_data: list) -> list:
    new_data = []

    for item in input_data:
        new_item = {key: val for key, val in item.items()}

        output = ask_model(item)
        new_item["output"] = '\n'.join(
            [distractor for distractor in output.splitlines() if distractor]
        )
        new_data.append(new_item)
    
    return new_data

In [12]:
new_data = get_model_answers(ege_eval_dataset)

In [13]:
new_data_df = pd.DataFrame(new_data)

In [15]:
new_data_df

,index,reading_text,question,right_answer,distractors,output
0,0,На задворках нашего села стояло на сваях длин...,Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?,Полонез вызвал у автора желание заплакать и с...,"[' Рассказчик был сиротой.', ' В детстве эта м...",Полонез побудил автора отправиться на войну.\n...
1,1,"Мы ехали берегом Лены на юг, а зима догоняла ...",Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?,"Подобравшись ближе к берегу, козы бросились к...","[' Собеседник рассказчика, Сокольский, сомнева...","Козы продвигались к нашему берегу, не обращая ..."
2,2,"Воспалённое состояние Поли, а главное, её сби...",Какое высказывание СООТВЕТСТВУЕТ тексту?,Автор письма хранит подарок девочки.,[' Родион встретил девочку перед наступлением....,Автор письма отправил подарок девочке обратно....
3,3,"Все мы любили «классного», хотя нельзя сказат...",Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?,Ребята сразу невзлюбили своего классного руко...,[' Белый билет не давал учителю возможности уй...,Классный руководитель всегда строго наказывал ...
4,4,В суровые военные годы во время бомбёжки моя ...,Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?,Бабушка рассказчицы долгое время работала в т...,"[' Убежище, в котором укрывалась рассказчица, ...",Бабушка рассказчицы была учительницей начальны...
5,6,В те отдалённые прежние времена приблизительн...,Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?,Человеческое общество за последние века дости...,"[' Одних знаний людям недостаточно, чтобы не с...",Люди современности окружены технологическими д...
6,7,"Когда Скоков пришёл к полному, тысячу раз про...",Какое высказывание СООТВЕТСТВУЕТ тексту?,За десять лет Скоков сумел накопить чуть мене...,[' Скоков занимал должность титулярного советн...,Скоков выполнил своё обещание и прожил сутки м...
7,8,Два камня У самого берега лежали два камня − ...,Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?,Старый Камень тоже мечтал вновь стать сильным...,"[' Чтобы стать кремнем, необходимо было попаст...",Старый Камень был доволен своей жизнью на бере...
8,9,В те отдалённые прежние времена приблизительн...,Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?,Человеческое общество за последние века дости...,"[' Одних знаний людям недостаточно, чтобы не с...",Человеческое общество за последнее время испыт...
9,10,"Эти трое были живые, смешливые, острые на язы...",Какое высказывание НЕ СООТВЕТСТВУЕТ тексту?,"Для того, чтобы человека уважали, недостаточн...",[' Молодые ребята произвели на рассказчика оче...,Ермаков был одним из самых честных и добросове...


In [19]:
new_data_df["distractors"] = new_data_df["distractors"].apply(
    lambda x: '\n'.join([distractor for distractor in literal_eval(x)])
)

In [ ]:
new_data_df.to_csv("../data_output_table/predict_distractors/Output_ChatGPT4o.csv", sep=';')

In [22]:
reading_text = """Наш корабль стоял на якоре у берега Африки. День был прекрасный, с моря дул свежий ветер; но к вечеру погода изменилась: стало душно и точно из топленной печки несло на нас горячим воздухом с пустыни Сахары. Перед закатом солнца капитан вышел на палубу, крикнул: «Купаться!» — и в одну минуту матросы попрыгали в воду, спустили в воду парус, привязали его и в парусе устроили купальню.
На корабле с нами было два мальчика. Мальчики первые попрыгали в воду, но им тесно было в парусе, они вздумали плавать наперегонки в открытом море.
Оба, как ящерицы, вытягивались в воде и что было силы поплыли к тому месту, где был бочонок над якорем.
Один мальчик сначала перегнал товарища, но потом стал отставать. Отец мальчика, старый артиллерист, стоял на палубе и любовался на своего сынишку. Когда сын стал отставать, отец крикнул ему: «Не выдавай! Понатужься!»
Вдруг с палубы кто-то крикнул: «Акула!» — и все мы увидали в воде спину морского чудовища.
Акула плыла прямо на мальчиков.
— Назад! Назад! Вернитесь! Акула! — закричал артиллерист. Но ребята не слыхали его, плыли дальше, смеялись и кричали еще веселее и громче прежнего.
Артиллерист, бледный как полотно, не шевелясь, смотрел на детей.
Матросы спустили лодку, бросились в нее и, сгибая весла, понеслись что было силы к мальчикам; но они были еще далеко от них, когда акула уже была не дальше 20-ти шагов.
Мальчики сначала не слыхали того, что им кричали, и не видали акулы; но потом один из них оглянулся, и мы все услыхали пронзительный визг, и мальчики поплыли в разные стороны.
Визг этот как будто разбудил артиллериста. Он сорвался с места и побежал к пушкам. Он повернул хобот, прилег к пушке, прицелился и взял фитиль.
Мы все, сколько нас ни было на корабле, замерли от страха и ждали, что будет.
Раздался выстрел, и мы увидали, что артиллерист упал подле пушки и закрыл лицо руками. Что сделалось с акулой и с мальчиками, мы не видали, потому что на минуту дым застлал нам глаза.
Но когда дым разошелся над водою, со всех сторон послышался сначала тихий ропот, потом ропот этот стал сильнее, и, наконец, со всех сторон раздался громкий, радостный крик.
Старый артиллерист открыл лицо, поднялся и посмотрел на море.
По волнам колыхалось желтое брюхо мертвой акулы. В несколько минут лодка подплыла к мальчикам и привезла их на корабль."""
question = "Какое высказывание СООТВЕТСТВУЕТ тексту?"
right_answer = "Меткость старого артиллериста спасла мальичков от морского чудовища"

In [23]:
arbitrary_item = {
    "reading_text": reading_text,
    "right_answer": right_answer,
    "question": question
}

In [25]:
distractors_a = get_model_answers([arbitrary_item])
distractors_a

[{'reading_text': 'Наш корабль стоял на якоре у берега Африки. День был прекрасный, с моря дул свежий ветер; но к вечеру погода изменилась: стало душно и точно из топленной печки несло на нас горячим воздухом с пустыни Сахары. Перед закатом солнца капитан вышел на палубу, крикнул: «Купаться!» — и в одну минуту матросы попрыгали в воду, спустили в воду парус, привязали его и в парусе устроили купальню.\nНа корабле с нами было два мальчика. Мальчики первые попрыгали в воду, но им тесно было в парусе, они вздумали плавать наперегонки в открытом море.\nОба, как ящерицы, вытягивались в воде и что было силы поплыли к тому месту, где был бочонок над якорем.\nОдин мальчик сначала перегнал товарища, но потом стал отставать. Отец мальчика, старый артиллерист, стоял на палубе и любовался на своего сынишку. Когда сын стал отставать, отец крикнул ему: «Не выдавай! Понатужься!»\nВдруг с палубы кто-то крикнул: «Акула!» — и все мы увидали в воде спину морского чудовища.\nАкула плыла прямо на мальчиков